# Prototype

This whole file assumes that the extension is from the bottom slice.
Will need to re-work to handle the more general case.

It is explicitly a prototype. This code will have to be re-addressed to make it work for another set of patient data.

## Description of method

This notebook takes a set of CT slices and a structure set file and appends extra CT slices to the bottom. To achieve this the following needs to be done:

* New UIDs need to be created for the new dicom slices
* New slice locations and patient position tags need to be created
* New CT instance numbers need to be created
* All new CT slice UIDs need to be added to the structure set file

Each relevant section is highlighted below.

## Initialisation

In [ ]:
import re
from copy import deepcopy
from glob import glob

import numpy as np

import pydicom

In [ ]:
y_coord_to_copy = -200
copy_up_till = -300  # won't include this position
copy_increments = -5

new_y_positons = np.arange(
    y_coord_to_copy + copy_increments, copy_up_till, copy_increments)

number_of_new_slices = len(new_y_positons)
new_y_positons

In [ ]:
ct_image_files_not_sorted = np.array(glob('../../../Scratch/extend-ct-patient-data/*image*.dcm'))
structure_set_file = glob('../../../Scratch/extend-ct-patient-data/*strctr*.dcm')[0]

In [ ]:
last_file_name = np.sort(ct_image_files_not_sorted)[-1]
re_result = re.match(r'(^.*_image)(\d\d\d\d\d)(\.DCM$)', last_file_name)
reference_filename_number = int(re_result.group(2))

new_filename_numbers = reference_filename_number + np.arange(0, number_of_new_slices) + 1
new_ct_slice_filenames = [
    ''.join([re_result.group(1), str(number).zfill(5), re_result.group(3)])
    for number in new_filename_numbers
]

new_ct_slice_filenames

In [ ]:
ct_image_dcm_not_sorted = np.array([
    pydicom.read_file(file, force=True)
    for file in ct_image_files_not_sorted
])

# ct_image_dcm

### Dicom tags for reference

In [ ]:
ct_image_dcm_not_sorted[0].dir()

### Loading data

In [ ]:
instance_numbers_not_sorted = np.array([
    dcm.InstanceNumber
    for dcm in ct_image_dcm_not_sorted
]).astype(int)

sorting_index = np.argsort(instance_numbers_not_sorted)

ct_image_files = ct_image_files_not_sorted[sorting_index]
ct_image_dcm = ct_image_dcm_not_sorted[sorting_index]

In [ ]:
uid = np.array([
    dcm.SOPInstanceUID
    for dcm in ct_image_dcm
])

## Finding the slice to be copied

In [ ]:
image_position = np.array([
    dcm.ImagePositionPatient
    for dcm in ct_image_dcm
])

y_coord = image_position[:, 2]
slice_to_copy_ref = y_coord == y_coord_to_copy
slice_to_copy = ct_image_dcm[slice_to_copy_ref][0]

## Creating the new UIDs

In [ ]:
original_uid = uid[slice_to_copy_ref][0]
original_uid_start = original_uid[0:-29]
original_uid_end = original_uid[-29::]

new_end_uid_numbers = np.arange(number_of_new_slices) + int(original_uid_end) + 1
new_uids = [
    ''.join([original_uid_start, str(number)])
    for number in new_end_uid_numbers
]

new_uids

## Creating the new slice locations

In [ ]:
new_slice_locations = list(new_y_positons.astype(str))
new_slice_locations

## Creating the new image positions

In [ ]:
reference_image_position = slice_to_copy.ImagePositionPatient

new_image_positions = [
    [
        str(reference_image_position[0]), 
        str(reference_image_position[1]), 
        str(y_pos)]
    for y_pos in new_y_positons
]

new_image_positions

## Creating the new instance numbers

In [ ]:
reference_instance_number = slice_to_copy.InstanceNumber

new_instance_numbers = list((
    int(slice_to_copy.InstanceNumber) + np.arange(len(new_y_positons)) + 1
).astype(str))

new_instance_numbers

## Creating the new CT slices

Deep copy must be used here otherwise all slices will end up the same as each other.

In [ ]:
new_slices = []

for uid, instance, slice_location, position in zip(
    new_uids, new_instance_numbers, new_slice_locations, new_image_positions
):
    new_slice = deepcopy(slice_to_copy)
    new_slice.SOPInstanceUID = uid
    new_slice.InstanceNumber = instance
    new_slice.SliceLocation = slice_location
    new_slice.ImagePositionPatient = position
    
    new_slices.append(new_slice)
    

## Updating the structure set dicom file with the new CT slices

In [ ]:
structure_dcm = pydicom.read_file(structure_set_file, force=True)
contour_image_sequence = structure_dcm.ReferencedFrameOfReferenceSequence[0].RTReferencedStudySequence[0].RTReferencedSeriesSequence[0].ContourImageSequence

for uid in new_uids:
    reference_dataset = deepcopy(contour_image_sequence[0])
    reference_dataset.ReferencedSOPInstanceUID = uid
    contour_image_sequence.append(reference_dataset)

    
contour_image_sequence

## Saving the new files

In [ ]:
for dcm, filename in zip(new_slices, new_ct_slice_filenames):
    dcm.save_as(filename=filename)

In [ ]:
structure_dcm.save_as(filename=structure_set_file)